In [22]:
import unittest
from datetime import datetime, timedelta
from db_controller import *

In [23]:
# Controller oluşturma
controller = DatabaseController(db_name="test.db")

In [24]:
controller = DatabaseController()

controller._cursor.executescript("""
        DROP TABLE IF EXISTS Book;
        DROP TABLE IF EXISTS Users;
        DROP TABLE IF EXISTS Comment;
        DROP TABLE IF EXISTS Reservation;
        DROP TABLE IF EXISTS Notification;
        DROP TABLE IF EXISTS ReadingList;
        
        CREATE TABLE Book (
            ISBN TEXT PRIMARY KEY,
            Title TEXT,
            Authors TEXT,
            Description TEXT,
            Genre TEXT,
            Availability INTEGER
        );
        CREATE TABLE Users (
            UserID INTEGER PRIMARY KEY AUTOINCREMENT,
            Name TEXT,
            FavouriteGenre TEXT,
            Password TEXT,
            Fine REAL
        );
        CREATE TABLE Comment (
            CommentID INTEGER PRIMARY KEY AUTOINCREMENT,
            UserID INTEGER,
            BookISBN TEXT,
            CommentText TEXT,
            CommentDate TEXT,
            FOREIGN KEY(UserID) REFERENCES Users(UserID),
            FOREIGN KEY(BookISBN) REFERENCES Book(ISBN)
        );
        CREATE TABLE Reservation (
            ReservationID INTEGER PRIMARY KEY AUTOINCREMENT,
            UserID INTEGER,
            BookISBN TEXT,
            ReservationDate TEXT,
            DueDate TEXT,
            Status TEXT,
            FOREIGN KEY(UserID) REFERENCES Users(UserID),
            FOREIGN KEY(BookISBN) REFERENCES Book(ISBN)
        );
        CREATE TABLE Notification (
            NotificationID INTEGER PRIMARY KEY AUTOINCREMENT,
            UserID INTEGER,
            Message TEXT,
            NotificationDate TEXT,
            FOREIGN KEY(UserID) REFERENCES Users(UserID)
        );
        CREATE TABLE ReadingList (
            ReadingListID INTEGER PRIMARY KEY AUTOINCREMENT,
            UserID INTEGER,
            BookISBN TEXT,
            FOREIGN KEY(UserID) REFERENCES Users(UserID),
            FOREIGN KEY(BookISBN) REFERENCES Book(ISBN)
        );
        """)

controller._conn.commit()

In [25]:
def test_add_book():
        result = controller.add_book(
            isbn='1234567890',
            title='Test',
            authors='Author One, Author Two',
            description='This is a test book.',
            genre='Fiction',
            availability=1
        )

        assert result is True, "Book addition failed"
        controller._cursor.execute("SELECT * FROM Book WHERE ISBN = ?", ('1234567890',))
        book = controller._cursor.fetchone()
        assert book is not None, "Book was not found in the database"

        print("test_add_book passed!")

test_add_book()

test_add_book passed!


In [26]:
def test_search_books():
    # Search by title
    results = controller.search_books(title='Test')
    assert len(results) == 1, "Search by title failed"
    print("Search by title passed!")
    
    # Search by author
    results = controller.search_books(author='Author One')
    assert len(results) == 1, "Search by author failed"
    print("Search by author passed!")
    
    # Search by genre
    results = controller.search_books(genre='Fiction')
    assert len(results) == 1, "Search by genre failed"
    print("Search by genre passed!")
    
    # Search with no results
    results = controller.search_books(title='Nonexistent')
    assert len(results) == 0, "Search with no results failed"
    print("Search with no results passed!")

test_search_books()

Search by title passed!
Search by author passed!
Search by genre passed!
Search with no results passed!


In [27]:
def test_add_user():
        result = controller.add_user(
            name='Test User',
            password='password123',
            favourite_genre='Fiction'
        )
        assert result is True, "User addition failed"
        # Verify the user was added
        controller._cursor.execute("SELECT * FROM Users WHERE Name = ?", ('Test User',))
        user = controller._cursor.fetchone()
        assert user is not None, "User was not found in the database"
        print("test_add_user passed!")
    
test_add_user()

test_add_user passed!


In [29]:
def test_add_comment():
        # Get user ID
        
        user_id = controller._cursor.execute("SELECT UserID FROM Users WHERE Name = ?", ('Test User',)).fetchone()[0]
        # Add comment
        result = controller.add_comment(
            user_id=user_id,
            book_isbn='1234567890',
            comment_text='Great book!'
        )
        assert result is True, "Comment addition failed"
        # Verify the comment was added
        controller._cursor.execute("SELECT * FROM Comment WHERE UserID = ? AND BookISBN = ?", (user_id, '1234567890'))
        comment = controller._cursor.fetchone()
        assert comment is not None, "Comment was not found in the database"
        print("test_add_comment passed!")

test_add_comment()

test_add_comment passed!


In [30]:
def test_update_fine():
        
        user_id = controller._cursor.execute("SELECT UserID FROM Users WHERE Name = ?", ('Test User',)).fetchone()[0]
        # Update fine
        result = controller.update_fine(user_id, 10.0)
        assert result is True, "Fine update failed"
        # Verify fine is updated
        controller._cursor.execute("SELECT Fine FROM Users WHERE UserID = ?", (user_id,))
        fine = controller._cursor.fetchone()[0]
        assert fine == 10.0, "Fine was not updated"
        print("test_update_fine passed!")

test_update_fine()

test_update_fine passed!


In [31]:
def test_create_book_report():
        # Add reservation
        controller._cursor.execute("SELECT UserID FROM Users WHERE Name = ?", ('Test User',))
        user_id = controller._cursor.fetchone()[0]
        due_date = datetime.now() + timedelta(days=7)
        result = controller.add_reservation(
            user_id=user_id,
            book_isbn='1234567890',
            due_date=due_date
        )
        assert result is True, "Reservation addition failed"
        
        # Add comment
        result = controller.add_comment(
            user_id=user_id,
            book_isbn='1234567890',
            comment_text='Great book!'
        )
        assert result is True, "Comment addition failed"
        
        # Create book report
        report = controller.createBookReport('1234567890')
        print(report)
        assert len(report) == 1, "Book report creation failed"
        report_data = report[0]
        assert (report_data[0] == 'Test Book'), "Incorrect book title in report" # Title
        assert (report_data[4] == 1), "Incorrect reservation count in report" # ReservationCount
        assert (report_data[5] == 1), "Incorrect comment count in report" # CommentCount
        print("test_create_book_report passed!")

test_create_book_report()

('Test', 'Author One, Author Two', 'This is a test book.', 'Fiction', 1, 2)


AssertionError: Book report creation failed

In [32]:
def test_create_user_report():

        controller._cursor.execute("SELECT UserID FROM Users WHERE Name = ?", ('Test User',))
        user_id = controller._cursor.fetchone()[0]
        due_date = datetime.now() + timedelta(days=7)
        controller.add_reservation(
            user_id=user_id,
            book_isbn='1234567890',
            due_date=due_date
        )
        # Create user report
        report = controller.createUserReport(user_id)
        assert len(report) == 1, "User report creation failed"
        report_data = report[0]
        assert (report_data[0]== 'Test User'),"Incorrect user name in report" # Name
        assert(report_data[2]== 1),"Incorrect reservation count in report"  # ReservationCount
        assert(report_data[3]== 1),"Incorrect comment count in report"  # CommentCount

        print("test_create_user_report passed!")

test_create_user_report()


Rapor oluşturma hatası: no such column: r.UserID


AssertionError: User report creation failed

In [33]:
controller._conn.close()
del controller